In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2

In [2]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.VQVAE_transformer import VQVAETransformer
from exit.motiontransformer import MotionTransformerOnly2, generate_src_mask
from exit.utils import get_model, visualize_2motions

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 'kit'
args.down_t = 2

train_loader_token = dataset_tokenize.DATALoader(args.dataname, 128, num_workers=1)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 128, w_vectorizer) # batch_size=128

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


100%|██████████| 300/300 [00:00<00:00, 3089.54it/s]


Pointer Pointing at 0
Reading checkpoints/kit/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!


In [4]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

In [5]:
args.nb_code = 512 # 8192 # 
args.code_dim = 128 #512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99
args.embed_dim_gpt = 1024
args.clip_dim = 512
args.block_size = 196+1
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4
args.vq_name = 'VQT_8_cbEmb512x128_fixQMultiGPU'
args.exp_name = 'TEMP'
args.out_dir = 'output_GPT_Final'
args.out_dir = os.path.join(args.out_dir, f'{args.exp_name}')
args.vq_dir= f'output/{args.vq_name}'
codebook_dir = f'{args.vq_dir}/codebook/'
os.makedirs(args.out_dir, exist_ok = True)
os.makedirs(args.vq_dir, exist_ok = True)
os.makedirs(codebook_dir, exist_ok = True)
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                            embed_dim=args.embed_dim_gpt, 
                                            clip_dim=args.clip_dim, 
                                            block_size=args.block_size, 
                                            num_layers=args.num_layers, 
                                            n_head=args.n_head_gpt, 
                                            drop_out_rate=args.drop_out_rate, 
                                            fc_rate=args.ff_rate)


trans_encoder = torch.nn.DataParallel(trans_encoder)
trans_encoder.train()
trans_encoder.cuda()
''

''

In [6]:
from exit.VQVAE_transformer import VQVAETransformer
net = VQVAETransformer(args).cuda()
net = torch.nn.DataParallel(net)
args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BiDirection/output/{args.vq_name}/net_last.pth'
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
net.eval()
net.cuda()
''

''

### VQVAE encoder index precomputed

In [24]:
for batch in train_loader_token:
    pose, name, m_length = batch
    pose = pose.cuda().float()
    src_mask = generate_src_mask(pose.shape[1], m_length).to(pose.device).unsqueeze(-1)
    pose = pose * src_mask
    
    target = net(pose, src_mask, type='encode')
    target = target.cpu().numpy()
    for i in range(target.shape[0]):
        trim_target = target[i:i+1][:m_length[i]]
        np.save(pjoin(codebook_dir, name[i] +'.npy'), trim_target)
blank_id = torch.mode(torch.tensor(target[:, -1])).values
blank_id = blank_id.item()

In [25]:
######## count used indices
# all_indices = None
# for batch in train_loader_token:
#     pose, name, m_length = batch
#     pose = pose.cuda().float()
#     src_mask = generate_src_mask(pose.shape[1], m_length).to(pose.device).unsqueeze(-1)
#     pose = pose * src_mask
    
#     target = net(pose, src_mask, type='encode')
#     if all_indices is None:
#         all_indices = target
#     else:
#         all_indices = torch.cat([all_indices, target])
# c = torch.bincount(all_indices.view(-1))
# (c==0).sum()

tensor(219, device='cuda:0')

### Train

In [ ]:
args.batch_size = 256 #1024 
num_workers = 1 # 8
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, codebook_dir, blank_id=blank_id, num_workers=num_workers)
train_loader_iter = dataset_TM_train.cycle(train_loader)

In [ ]:
batch = next(train_loader_iter)

In [30]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args, **kwargs):
        return
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')


test
./Test/FID fid nb_iter


In [15]:
args.pkeep = .5
batch = next(train_loader_iter)
clip_text, m_tokens, m_tokens_len = batch
m_tokens, m_tokens_len = m_tokens.cuda(), m_tokens_len.cuda()
bs = m_tokens.shape[0]
target = m_tokens    # (bs, 26)
target = target.cuda()

text = clip.tokenize(clip_text, truncate=True).cuda()
    
feat_clip_text = clip_model.encode_text(text).float()

input_index = target[:,:-1]

if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(input_index.shape,
                                                        device=input_index.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(input_index.shape,
                                                        device=input_index.device))
mask = mask.round().to(dtype=torch.int64)
r_indices = torch.randint_like(input_index, args.nb_code)
a_indices = mask*input_index+(1-mask)*r_indices

cls_pred = trans_encoder(a_indices, feat_clip_text)
cls_pred = cls_pred.contiguous()
cls_pred.shape

StopIteration: 

In [ ]:
args.if_maxtest = False

right_num = 0
loss_ce = torch.nn.CrossEntropyLoss()
loss_cls = 0.0
for i in range(bs):
    # loss function     (26), (26, 513)
    loss_cls += loss_ce(cls_pred[i][:m_tokens_len[i] + 1], target[i][:m_tokens_len[i] + 1]) / bs

    # Accuracy
    probs = torch.softmax(cls_pred[i][:m_tokens_len[i] + 1], dim=-1)

    if args.if_maxtest:
        _, cls_pred_index = torch.max(probs, dim=-1)
    else:
        dist = Categorical(probs)
        cls_pred_index = dist.sample()
    right_num += (cls_pred_index.flatten(0) == target[i][:m_tokens_len[i] + 1].flatten(0)).sum().item()

### Eval

In [33]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 300/300 [00:00<00:00, 1731.82it/s]

Pointer Pointing at 0


In [34]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [36]:
batch = next(iter(val_loader))
word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = batch
pose = pose.cuda().float()
bs, seq = pose.shape[:2]
num_joints = 21 if pose.shape[-1] == 251 else 22
text = clip.tokenize(clip_text, truncate=True).cuda()

feat_clip_text = clip_model.encode_text(text).float()
src_mask = generate_src_mask(pose.shape[1], m_length).to(pose.device).unsqueeze(-1)
index_motion = trans_encoder(feat_clip_text, False, type="sample")

In [9]:
index_motion = torch.randint(0, 8192, (128, 196)).cuda()
# blank_id = 88 # get_model(trans_encoder).num_vq
index_motion[1, 5] = blank_id
index_motion[1, 7] = blank_id
index_motion[4, 9] = blank_id

# l_idx, l_val = (index_motion == blank_id).nonzero(as_tuple=True)
lengths = torch.ones(index_motion.shape[0], device=index_motion.device) * index_motion.shape[1]
pred_length = (index_motion == blank_id).int().argmax(1).float()
lengths[pred_length>0] = pred_length[pred_length>0]
src_mask = generate_src_mask(index_motion.shape[1], lengths)
index_motion[~src_mask] = blank_id
src_mask = src_mask.unsqueeze(-1)

In [ ]:
### Run full eval (slow) ###
best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = \
    eval_trans.evaluation_transformer(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        eval_wrapper=eval_wrapper)

In [ ]:
pred_pose_eval, pose, m_length, clip_text, best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = eval_trans.evaluation_transformer(args.out_dir, val_loader, net, trans_encoder, logger, writer, 0, best_fid=1000, best_iter=0, best_div=100, best_top1=0, best_top2=0, best_top3=0, best_matching=100, clip_model=clip_model, eval_wrapper=eval_wrapper)

In [ ]:
x = pose[0].detach().cpu().numpy()
y = pred_pose_eval[0].detach().cpu().numpy()
visualize_2motions(x, val_loader.dataset.std, val_loader.dataset.mean, args.dataname, m_length[0], y)